# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   index                  15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  mau                    15000 non-null  bool   
 15  re

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

df = pd.read_csv('previsao_de_renda.csv')

df['data_ref'] = pd.to_datetime(df['data_ref'])

df['log_renda'] = np.log(df['renda'])

categorical_cols = df.select_dtypes(include='object').columns.tolist()

df_dummies = pd.get_dummies(df[categorical_cols], drop_first=True)

df_model = pd.concat([df, df_dummies], axis=1)


df_model = df_model.drop(columns=categorical_cols)

# Renomear as colunas problemáticas para evitar erros de sintaxe
def sanitize_column_name(name):
    return ''.join(c if c.isalnum() else '_' for c in name)

df_model.columns = [sanitize_column_name(col) for col in df_model.columns]

formula_inicial = 'log_renda ~ ' + '+'.join(df_model.columns.drop(['log_renda', 'data_ref', 'index', 'renda']))
modelo_inicial = smf.ols(formula_inicial, data=df_model).fit()

print("Modelo Inicial:")
print(modelo_inicial.summary())

def avaliar_modelo(modelo, nome="Modelo"):
    print(f"\n{nome}:")
    print(modelo.summary())
    print(f"AIC: {modelo.aic:.2f}")
    print(f"BIC: {modelo.bic:.2f}")

avaliar_modelo(modelo_inicial, "Modelo Inicial")

modelo_atual = modelo_inicial
historico_modelos = [modelo_inicial]

while True:
    p_values = modelo_atual.pvalues.drop('Intercept')
    if p_values.empty:
        print("\nNão há mais variáveis para remover.")
        modelo_final = modelo_atual
        break
    variavel_menos_significante = p_values.idxmax()
    p_value_max = p_values.max()

    if p_value_max > 0.05:
        formula_atual = modelo_atual.model.formula.replace(f'+{variavel_menos_significante}', '').replace(variavel_menos_significante, '')

        if formula_atual.count('~') == 1 and formula_atual.split('~')[1].strip() == '':
            print("\nParando a remoção de variáveis para evitar modelo sem preditores.")
            modelo_final = modelo_atual
            break

        try:
            modelo_atual = smf.ols(formula_atual, data=df_model).fit()
            historico_modelos.append(modelo_atual)
            print(f"\nRemovendo a variável: {variavel_menos_significante} (p-value: {p_value_max:.3f})")
            avaliar_modelo(modelo_atual, "Modelo Atual")
        except Exception as e:
            print(f"\nErro ao ajustar o modelo após remover {variavel_menos_significante}: {e}")
            modelo_final = historico_modelos[-1] # Reverte para o modelo anterior
            break
    else:
        print("\nNenhuma variável com p-value > 0.05 encontrada. Modelo final obtido.")
        modelo_final = modelo_atual
        break

print("\nComparação do Modelo Final com o Modelo Inicial:")
avaliar_modelo(modelo_inicial, "Modelo Inicial")
avaliar_modelo(modelo_final, "Modelo Final")

